In [13]:
import os
import pickle
import sys

import ase
import ase.data
import ase.io
import ase.visualize
import jax
import jax.numpy as jnp
import jraph
import ml_collections
import numpy as np
import tqdm
import yaml

from typing import List

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

sys.path.append('..')

In [6]:
import datatypes
import input_pipeline
import qm9
from analyses import analysis
from models import ATOMIC_NUMBERS, RADII, create_model

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [7]:
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/mace/interactions=3/l=2/channels=32"
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/e3schnet/interactions=4/l=3/channels=32"
# path = "/home/ameyad/spherical-harmonic-net/workdirs/v3/mace/interactions=4/l=3/channels=32"
path = "/Users/songk/atomicarchitects/spherical_harmonic_net/workdirs/v3/mace/interactions=4/l=5/channels=32"

with open(path + "/checkpoints/params.pkl", 'rb') as f:
    params = pickle.load(f)
with open(path + "/config.yml", "rt") as config_file:
    config = yaml.unsafe_load(config_file)

assert config is not None
config = ml_collections.ConfigDict(config)

In [8]:
config.target_position_predictor.res_alpha = 359
config.target_position_predictor.res_beta = 180

config

activation: softplus
avg_num_neighbors: 15.0
checkpoint_every_steps: 1000
eval_every_steps: 1000
focus_predictor:
  latent_size: 128
  num_layers: 2
learning_rate: 0.001
log_every_steps: 1000
loss_kwargs:
  radius_rbf_variance: 0.001
max_ell: 5
max_n_edges: 1024
max_n_graphs: 64
max_n_nodes: 512
model: MACE
nn_cutoff: 5.0
nn_tolerance: 0.5
num_basis_fns: 8
num_channels: 32
num_eval_steps: 100
num_eval_steps_at_end_of_training: 5000
num_interactions: 4
num_species: 5
num_train_steps: 20000
optimizer: adam
r_max: 5
rng_seed: 0
root_dir: /Users/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/data_tf2
target_position_predictor:
  res_alpha: 359
  res_beta: 180
target_species_predictor:
  latent_size: 128
  num_layers: 2
test_molecules: !!python/tuple
- 53568
- 133920
train_molecules: !!python/tuple
- 0
- 47616
val_molecules: !!python/tuple
- 47616
- 53568

In [9]:
model = create_model(config, run_in_evaluation_mode=True)
apply_fn = jax.jit(model.apply)

def apply(frag, seed, beta):
    frags = jraph.pad_with_graphs(frag, 32, 1024, 2)
    preds = apply_fn(params, seed, frags, beta)
    pred = jraph.unpad_with_graphs(preds)
    return pred

In [10]:
def append_pred_to_ase_atoms(molecule: ase.Atoms, pred: datatypes.Predictions) -> ase.Atoms:
    focus = pred.globals.focus_indices.squeeze(0)
    pos_focus = molecule.positions[focus]
    pos_rel = pred.globals.position_vectors.squeeze(0)

    new_specie = jnp.array(
        ATOMIC_NUMBERS[pred.globals.target_species.squeeze(0).item()]
    )
    new_position = pos_focus + pos_rel

    return ase.Atoms(
        positions=jnp.concatenate([molecule.positions, new_position[None, :]], axis=0),
        numbers=jnp.concatenate([molecule.numbers, new_specie[None]], axis=0),
    )


In [11]:
name = path.split("/")[-5:]
name = "_".join(name)

beta = 1

name = f"{name}_beta={beta}"

print(name)

v3_mace_interactions=4_l=5_channels=32_beta=1


In [12]:
molecules = []

if not os.path.exists(f"gen/{name}"):
    os.mkdir(f"gen/{name}")

for seed in tqdm.tqdm(range(64)):
    molecule = ase.Atoms(
        positions=jnp.array([[0, 0, 0.0]]),
        numbers=jnp.array([6]),
    )

    rng = jax.random.PRNGKey(seed)
    for step in range(31):
        k, rng = jax.random.split(rng)
        frag = input_pipeline.ase_atoms_to_jraph_graph(
            molecule, ATOMIC_NUMBERS, config.nn_cutoff
        )
        pred = apply(frag, k, beta)

        stop = pred.globals.stop.squeeze(0).item()
        if stop:
            molecules.append(molecule)

            ase.io.write(f"gen/{name}/molecule_{seed}.xyz", molecule)

            print(f"Generated molecule {seed} of {len(molecule)} atoms")
            break

        molecule = append_pred_to_ase_atoms(molecule, pred)


  2%|▏         | 1/64 [02:34<2:42:35, 154.86s/it]

Generated molecule 0 of 15 atoms


  3%|▎         | 2/64 [02:56<1:19:00, 76.45s/it] 

Generated molecule 1 of 14 atoms


  5%|▍         | 3/64 [03:10<48:40, 47.88s/it]  

...

Generated molecule 61 of 21 atoms


 98%|█████████▊| 63/64 [46:18<00:30, 30.82s/it]

Generated molecule 62 of 13 atoms


100%|██████████| 64/64 [46:47<00:00, 43.87s/it]

Generated molecule 63 of 18 atoms


In [14]:
def ase_to_mol_dict(molecules: List[ase.Atoms], save=True, model_path=None, file_name=None):
    '''from G-SchNet: https://github.com/atomistic-machine-learning/G-SchNet'''

    generated = (
        {}
    )
    for mol in molecules:
        l = mol.get_atomic_numbers().shape[0]
        if l not in generated:
            generated[l] = {
                "_positions": np.array([mol.get_positions()]),
                "_atomic_numbers": np.array([mol.get_atomic_numbers()]),
            }
        else:
            generated[l]["_positions"] = np.append(
                generated[l]["_positions"],
                np.array([mol.get_positions()]),
                0,
            )
            generated[l]["_atomic_numbers"] = np.append(
                generated[l]["_atomic_numbers"],
                np.array([mol.get_atomic_numbers()]),
                0,
            )

    return generated

In [15]:
name

'v3_mace_interactions=4_l=5_channels=32_beta=1'

In [16]:
with open(f'gen/{name}/molecules.pkl', 'wb') as f:
    pickle.dump(molecules, f)

In [17]:
mol_dict = ase_to_mol_dict(molecules)

In [20]:
with open("../workdirs/generated/generated_molecules.mol_dict", "wb") as f:
    pickle.dump(mol_dict, f)